In [1]:
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

Using TensorFlow backend.
/Users/jackholder/anaconda2/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
numpy.random.seed(10)

In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170434560/170498071 [============================>.] - ETA: 0s

In [4]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [5]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [6]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 16, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               4194816   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
__________

In [7]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32)

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 579s - loss: 1.7179 - acc: 0.3760 - val_loss: 1.4273 - val_acc: 0.4793
Epoch 2/25
50000/50000 [==============================] - 557s - loss: 1.3367 - acc: 0.5192 - val_loss: 1.1635 - val_acc: 0.5842
Epoch 3/25
50000/50000 [==============================] - 561s - loss: 1.1654 - acc: 0.5833 - val_loss: 1.1009 - val_acc: 0.6042
Epoch 4/25
50000/50000 [==============================] - 557s - loss: 1.0463 - acc: 0.6253 - val_loss: 1.0499 - val_acc: 0.6286
Epoch 5/25
50000/50000 [==============================] - 560s - loss: 0.9525 - acc: 0.6625 - val_loss: 0.9867 - val_acc: 0.6487
Epoch 6/25
50000/50000 [==============================] - 560s - loss: 0.8720 - acc: 0.6907 - val_loss: 0.9716 - val_acc: 0.6572
Epoch 7/25
50000/50000 [==============================] - 565s - loss: 0.8057 - acc: 0.7137 - val_loss: 0.9354 - val_acc: 0.6721
Epoch 8/25
50000/50000 [=======================

In [8]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 69.88%
